In [1]:
# Get Product Details and Reviews from Amazon Products
# by: Sebastian Van Hemert
# reference: https://rapidapi.com/letscrape-6bRBa3QguO5/api/real-time-amazon-data/playground/apiendpoint_6e797385-80a3-4d2d-9f15-2a659278eaa4
# tutorial followed: https://rapidapi.com/blog/amazon-product-reviews-api/

#to-do:
#helpful_vote_statement:"9 people found this helpful"

In [2]:
!pip install IPython

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from IPython.display import Javascript
Javascript("Jupyter.notebook.kernel.restart({ 'timeout': 18000 })")

<IPython.core.display.Javascript object>

In [1]:
# Step 1: Declare the import statements and globals
import sys
import json
import requests
import time
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

RAPIDAPI_KEY  = "e205fd7dcbmsh9458a15a00fb8ebp123bcbjsn4d4f5f353fc0"

In [2]:
url = "https://real-time-amazon-data.p.rapidapi.com/product-reviews"

headers = {
    "x-rapidapi-key": RAPIDAPI_KEY,
    "x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
}

In [3]:
# import ASIN csv and convert to list
asin_df = pd.read_csv('asin_list.csv')
asin_list = asin_df['ASIN'].astype(str).tolist()
print(len(asin_list))
print()
print(asin_list)

334

['B0CN8KSFF2', 'B07R67X37H', 'B08WJR1W7F', 'B07FZTMCFH', 'B0813DJTGT', 'B09QH3FG2R', 'B0D8Z494NM', 'B0CLPJDW6G', 'B00JUM78S6', 'B09ZR4SGC8', 'B094Z42SGL', 'B073WTW5FM', 'B085BJ4MLK', 'B097CZ55H9', 'B07RD1497C', 'B0819W9CSW', 'B07XFLT4TS', 'B07DSBVQ2N', 'B08KXQFV1G', 'B073Q3FK8B', 'B0BNPT9DTZ', 'B07DNKZKCL', 'B08M9M9SMZ', 'B011LC85PM', 'B07XZNRR7Q', 'B084R7V46M', 'B072WHQFJ7', 'B01JJ0JTNK', 'B0BQNB41MK', 'B0BN4KBC48', 'B09JK3FPF3', 'B07FSSMNQ8', 'B0BC151Q4V', 'B09KM2T29B', 'B01MR9D7VJ', 'B0BM8G3W11', 'B0CP8CNWPX', 'B09QCMJQR9', 'B073H8STZ7', 'B00M2702JI', 'B07SXHC2PW', 'B09PDPGBQ3', 'B071HSTPDV', 'B0BNT3CXB1', 'B0CJHNV2LZ', 'B07WWJ5NDW', 'B07DKQGGW4', 'B09HGQG8KY', 'B07ZGMV9G9', 'B08B4PJNP6', 'B0CCZ26B5V', 'B09LYF2ST7', 'B00NJ2M33I', 'B0BWY8C7TQ', 'B07D4RLZ79', 'B0CLFD1K7V', 'nan', 'B0018MQAOY', 'B08HSS37H7', '\u200eB00JES3MO0', 'B0BXHJ3873', 'nan', 'B0BW4KF65K', 'nan', 'nan', 'B00JVTNXIC', 'B0BXMNC25J', 'B00F3H79LS', 'B0CYL8HG1F', 'B0C1HCCK2T', 'B071XHP12B', 'B00PC5CI8K', 'B096ZD7

In [4]:
# Initialize an empty DataFrame to store all reviews
all_reviews = pd.DataFrame()

# Use tqdm to show progress bar
for asin in tqdm(asin_list, desc="Fetching reviews"):
    querystring = {
        "asin": asin,
        "country": "US",
        "sort_by": "TOP_REVIEWS",
        "star_rating": "ALL",
        "verified_purchases_only": "false"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        reviews = response.json()['data']['reviews']
        df = pd.DataFrame(reviews)
        df['asin'] = asin
        all_reviews = pd.concat([all_reviews, df], ignore_index=True)
    else:
        print(f"Failed to retrieve data for ASIN {asin}: {response.status_code} - {response.text}")

print(all_reviews.head())

Fetching reviews: 100%|██████████| 334/334 [05:18<00:00,  1.05it/s]

        review_id                               review_title  \
0   RU2LN4GFJTIOA  Great personalized pens at a great price!   
1   R7JKFNZ7GKUZ9         Beautiful Pens, more than expected   
2  R1BMFBC78IWMEV                        Great for the price   
3   RA2Z63BT6RUSZ                                  Overjoyed   
4  R21W2492GLLIT8                                    Garbage   

                                      review_comment review_star_rating  \
0  Much nicer than expected, very smooth when wri...                  5   
1  The pen is beautiful.  The customize lettering...                  5   
2  I am still getting my business off the ground ...                  5   
3  as a part of my daughter-in-law‘s birthday gif...                  5   
4  They are cute but the actual pen breaks so easily                  1   

                                         review_link    review_author  \
0  https://www.amazon.com/gp/customer-reviews/RU2...    Jenny Sheline   
1  https://www.ama

In [5]:
print(len(all_reviews))
print(all_reviews.columns)

3108
Index(['review_id', 'review_title', 'review_comment', 'review_star_rating',
       'review_link', 'review_author', 'review_author_avatar', 'review_images',
       'review_video', 'review_date', 'is_verified_purchase',
       'reviewed_product_asin', 'helpful_vote_statement', 'asin'],
      dtype='object')


In [6]:
print(all_reviews.shape)

# Remove duplicates from the combined dataframe
all_reviews.drop_duplicates(subset=['review_id'], inplace=True)

print(all_reviews.shape)

(3108, 14)
(3088, 14)


In [7]:
# List of null-like values to filter out
null_like_values = [None, np.nan, "NULL", "", '', '0']

all_reviews_with_nulls = all_reviews.copy()

# Filter rows where reviewId is in null_like_values
all_reviews = all_reviews[~all_reviews['review_id'].isin(null_like_values)]

# Optionally, if you want to modify the original DataFrame inplace:
# all_reviews = all_reviews[~all_reviews['reviewId'].isin(null_like_values)]

# Print number of rows removed
rows_removed = len(all_reviews_with_nulls) - len(all_reviews)
print(f"Removed {rows_removed} rows with null-like values in 'review_id' column.")

# Now 'filtered_reviews' contains rows where 'reviewId' is not null-like

Removed 0 rows with null-like values in 'review_id' column.


In [8]:
# Display the first few rows of the dataframe to inspect the 'date' column
print(all_reviews['review_date'].head())

# Remove the unnecessary text and keep only the date part
all_reviews['date_cleaned'] = all_reviews['review_date'].str.replace(r'Reviewed in .* on ', '', regex=True)

# Convert the cleaned date strings to datetime objects
all_reviews['date_cleaned'] = pd.to_datetime(all_reviews['date_cleaned'])

# Verify the conversion by displaying the first few rows
print(all_reviews[['review_date', 'date_cleaned']].head())

0    Reviewed in the United States on July 10, 2024
1    Reviewed in the United States on June 25, 2024
2     Reviewed in the United States on May 24, 2024
3    Reviewed in the United States on June 12, 2024
4      Reviewed in the United States on May 8, 2024
Name: review_date, dtype: object
                                      review_date date_cleaned
0  Reviewed in the United States on July 10, 2024   2024-07-10
1  Reviewed in the United States on June 25, 2024   2024-06-25
2   Reviewed in the United States on May 24, 2024   2024-05-24
3  Reviewed in the United States on June 12, 2024   2024-06-12
4    Reviewed in the United States on May 8, 2024   2024-05-08


In [9]:
# Example handling errors
all_reviews['review_star_rating'] = pd.to_numeric(df['review_star_rating'], errors='coerce')  # coerce errors to NaN

In [10]:
# Add placeholder empty column to match Richard's Amazon data
all_reviews['keyword'] = None

In [11]:
# Columns to delete
columns_to_keep = ['review_id', 'reviewed_product_asin', 'date_cleaned',
                   'review_author', 'review_title', 'keyword',
                   'is_verified_purchase', 'review_star_rating', 'review_comment']

# Drop the specified columns from the DataFrame
all_reviews = all_reviews[columns_to_keep]

# Extract the part before '?'
all_reviews['reviewed_product_asin'] = all_reviews['reviewed_product_asin'].str.split('?').str[0]

all_reviews

,review_id,reviewed_product_asin,date_cleaned,review_author,review_title,keyword,is_verified_purchase,review_star_rating,review_comment
0,RU2LN4GFJTIOA,B0CN8KSFF2,2024-07-10,Jenny Sheline,Great personalized pens at a great price!,None,True,5.0,"Much nicer than expected, very smooth when wri..."
1,R7JKFNZ7GKUZ9,B0CN8KSFF2,2024-06-25,Amazon Customer,"Beautiful Pens, more than expected",None,True,5.0,The pen is beautiful. The customize lettering...
2,R1BMFBC78IWMEV,B0CN8KSFF2,2024-05-24,Maria,Great for the price,None,True,5.0,I am still getting my business off the ground ...
3,RA2Z63BT6RUSZ,B0CN8KSFF2,2024-06-12,Princess,Overjoyed,None,True,4.0,as a part of my daughter-in-law‘s birthday gif...
4,R21W2492GLLIT8,B0CN8KSFF2,2024-05-08,chrissy,Garbage,None,True,5.0,They are cute but the actual pen breaks so easily
...,...,...,...,...,...,...,...,...,...
3103,R3V889SFLL9UGW,B005CVXFP4,2023-04-15,Sara leonard,Wrist bands,None,True,NaN,Used for St. Patty Day party. Worked like they...
3104,R3DC73CMBHYGN3,B005CVWZD2,2022-10-26,A'me M.,Perfect for our MEAD event,None,True,NaN,Worked great for age verified bands for our on...
3105,R2W0HZT4NG8GS0,B005CVXFP4,2019-09-19,bobwrong,Exactly what I needed,None,True,NaN,I use over 21 bracelets every year for an even...
3106,R3JDFJDBHU6B3W,B06XS3RTB5,2023-04-25,Carmen,Very easy to take off,None,True,NaN,They are good for the entrance but not for alc...


In [12]:
# Rename columns to match Richard's data
all_reviews = all_reviews.rename(columns={
    'review_id': 'reviewId',
    'reviewed_product_asin': 'asin',
    'date_cleaned': 'date_cleaned',
    'review_author': 'username',
    'review_title': 'title',
    'is_verified_purchase': 'verified',
    'review_star_rating': 'rating_cleaned',
    'review_comment': 'text'
})

all_reviews

,reviewId,asin,date_cleaned,username,title,keyword,verified,rating_cleaned,text
0,RU2LN4GFJTIOA,B0CN8KSFF2,2024-07-10,Jenny Sheline,Great personalized pens at a great price!,None,True,5.0,"Much nicer than expected, very smooth when wri..."
1,R7JKFNZ7GKUZ9,B0CN8KSFF2,2024-06-25,Amazon Customer,"Beautiful Pens, more than expected",None,True,5.0,The pen is beautiful. The customize lettering...
2,R1BMFBC78IWMEV,B0CN8KSFF2,2024-05-24,Maria,Great for the price,None,True,5.0,I am still getting my business off the ground ...
3,RA2Z63BT6RUSZ,B0CN8KSFF2,2024-06-12,Princess,Overjoyed,None,True,4.0,as a part of my daughter-in-law‘s birthday gif...
4,R21W2492GLLIT8,B0CN8KSFF2,2024-05-08,chrissy,Garbage,None,True,5.0,They are cute but the actual pen breaks so easily
...,...,...,...,...,...,...,...,...,...
3103,R3V889SFLL9UGW,B005CVXFP4,2023-04-15,Sara leonard,Wrist bands,None,True,NaN,Used for St. Patty Day party. Worked like they...
3104,R3DC73CMBHYGN3,B005CVWZD2,2022-10-26,A'me M.,Perfect for our MEAD event,None,True,NaN,Worked great for age verified bands for our on...
3105,R2W0HZT4NG8GS0,B005CVXFP4,2019-09-19,bobwrong,Exactly what I needed,None,True,NaN,I use over 21 bracelets every year for an even...
3106,R3JDFJDBHU6B3W,B06XS3RTB5,2023-04-25,Carmen,Very easy to take off,None,True,NaN,They are good for the entrance but not for alc...


In [13]:
# Column that tags whether subset is train or test
all_reviews['train_test'] = 'test'

# Verify the changes
all_reviews

,reviewId,asin,date_cleaned,username,title,keyword,verified,rating_cleaned,text,train_test
0,RU2LN4GFJTIOA,B0CN8KSFF2,2024-07-10,Jenny Sheline,Great personalized pens at a great price!,None,True,5.0,"Much nicer than expected, very smooth when wri...",test
1,R7JKFNZ7GKUZ9,B0CN8KSFF2,2024-06-25,Amazon Customer,"Beautiful Pens, more than expected",None,True,5.0,The pen is beautiful. The customize lettering...,test
2,R1BMFBC78IWMEV,B0CN8KSFF2,2024-05-24,Maria,Great for the price,None,True,5.0,I am still getting my business off the ground ...,test
3,RA2Z63BT6RUSZ,B0CN8KSFF2,2024-06-12,Princess,Overjoyed,None,True,4.0,as a part of my daughter-in-law‘s birthday gif...,test
4,R21W2492GLLIT8,B0CN8KSFF2,2024-05-08,chrissy,Garbage,None,True,5.0,They are cute but the actual pen breaks so easily,test
...,...,...,...,...,...,...,...,...,...,...
3103,R3V889SFLL9UGW,B005CVXFP4,2023-04-15,Sara leonard,Wrist bands,None,True,NaN,Used for St. Patty Day party. Worked like they...,test
3104,R3DC73CMBHYGN3,B005CVWZD2,2022-10-26,A'me M.,Perfect for our MEAD event,None,True,NaN,Worked great for age verified bands for our on...,test
3105,R2W0HZT4NG8GS0,B005CVXFP4,2019-09-19,bobwrong,Exactly what I needed,None,True,NaN,I use over 21 bracelets every year for an even...,test
3106,R3JDFJDBHU6B3W,B06XS3RTB5,2023-04-25,Carmen,Very easy to take off,None,True,NaN,They are good for the entrance but not for alc...,test


In [14]:
# Column that tags whether subset is train or test
all_reviews['data_source'] = 'new data'

# Verify the changes
all_reviews

,reviewId,asin,date_cleaned,username,title,keyword,verified,rating_cleaned,text,train_test,data_source
0,RU2LN4GFJTIOA,B0CN8KSFF2,2024-07-10,Jenny Sheline,Great personalized pens at a great price!,None,True,5.0,"Much nicer than expected, very smooth when wri...",test,new data
1,R7JKFNZ7GKUZ9,B0CN8KSFF2,2024-06-25,Amazon Customer,"Beautiful Pens, more than expected",None,True,5.0,The pen is beautiful. The customize lettering...,test,new data
2,R1BMFBC78IWMEV,B0CN8KSFF2,2024-05-24,Maria,Great for the price,None,True,5.0,I am still getting my business off the ground ...,test,new data
3,RA2Z63BT6RUSZ,B0CN8KSFF2,2024-06-12,Princess,Overjoyed,None,True,4.0,as a part of my daughter-in-law‘s birthday gif...,test,new data
4,R21W2492GLLIT8,B0CN8KSFF2,2024-05-08,chrissy,Garbage,None,True,5.0,They are cute but the actual pen breaks so easily,test,new data
...,...,...,...,...,...,...,...,...,...,...,...
3103,R3V889SFLL9UGW,B005CVXFP4,2023-04-15,Sara leonard,Wrist bands,None,True,NaN,Used for St. Patty Day party. Worked like they...,test,new data
3104,R3DC73CMBHYGN3,B005CVWZD2,2022-10-26,A'me M.,Perfect for our MEAD event,None,True,NaN,Worked great for age verified bands for our on...,test,new data
3105,R2W0HZT4NG8GS0,B005CVXFP4,2019-09-19,bobwrong,Exactly what I needed,None,True,NaN,I use over 21 bracelets every year for an even...,test,new data
3106,R3JDFJDBHU6B3W,B06XS3RTB5,2023-04-25,Carmen,Very easy to take off,None,True,NaN,They are good for the entrance but not for alc...,test,new data


In [15]:
# Export the combined dataframe to a CSV file
all_reviews.to_csv('processed_reviews_new_API_data.csv', index=False)